In [2]:
import requests
import pandas as pd

import urllib.request
import json
import os
import ssl

In [97]:
class API_values:
    
    def __init__(self):
        self.allowSelfSignedHttps(True) 
    
    url = 'https://workspace-fertilite-jxvfk.southafricanorth.inference.ml.azure.com/score'
    api_key = "wGiKQhKXs5YazqwkcpkBtpoGfX3JMOZE"
    azureml_model_deployment = 'automl4f78aa81750-1'
    
    def allowSelfSignedHttps(self, allowed):
        # bypass the server certificate verification on client side
        if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
            ssl._create_default_https_context = ssl._create_unverified_context
    
    def getPred(self, data):
        #self.allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.
        # Request data goes here
        # The example below assumes JSON formatting which may be updated
        # depending on the format your endpoint expects.
        # More information can be found here:
        # https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
        data_processed = self.transform(data).to_dict(orient='records') 
        content =  {
          "Inputs": {
            "data": [{"Season":0,"Childish diseases":1,"Accident or serious trauma":1,"Surgical intervention":1,"High fevers in the last year":2,"Frequency of alcohol consumption":1,"Smoking habit":2,"Age":30,"Number of hours spent sitting per day":8}]
            #"data": data_processed #[{"Season":0,"Childish diseases":1,"Accident or serious trauma":1,"Surgical intervention":1,"High fevers in the last year":2,"Frequency of alcohol consumption":1,"Smoking habit":2,"Age":30,"Number of hours spent sitting per day":8}]#data_processed.to_json(orient='records')
          },
          "GlobalParameters": {
            "method": "predict"
          }
        }

        body = str.encode(json.dumps(content))
        headers = {'Content-Type':'application/json', 
                   'Authorization':('Bearer '+ self.api_key), 
                   'azureml-model-deployment': self.azureml_model_deployment }
        req = urllib.request.Request(self.url, body, headers)
        result = None
        try:
            response = urllib.request.urlopen(req)
            result = response.read()
            print(result)
        except urllib.error.HTTPError as error:
            print("The request failed with status code: " + str(error.code))
            result = str(error.code)
            # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
            print(error.info())
            print(error.read().decode("utf8", 'ignore'))
        return result

    def transform(self,data):
        data['Childish diseases'] = data['Childish diseases'].map({'yes': 1, 'no': 0})
        data['Accident or serious trauma'] = data['Accident or serious trauma'].map({'yes': 1, 'no': 0})
        data['Surgical intervention'] = data['Surgical intervention'].map({'yes': 1, 'no': 0})
        data['High fevers in the last year'] = data['High fevers in the last year'].map({
            'more than 3 months ago': 2,
            'less than 3 months ago': 1,
            'no': 0
        })
        #data['Diagnosis'] = data['Diagnosis'].map({'Normal': 0, 'Altered': 1})
        data['Frequency of alcohol consumption'] = data['Frequency of alcohol consumption'].map({
            'once a week': 1,
            'several times a week': 2,
            'hardly ever or never': 0,
            'every day': 3
        })
        data['Season'] = data['Season'].map({
            'spring': 0,
            'fall': 1,
            'winter': 2,
            'summer': 3
        })
        data['Smoking habit'] = data['Smoking habit'].map({
            'occasional': 0,
            'daily': 1,
            'never': 2,
        })
        return data
    
data = {
    "Season": ["spring"],
    "Childish diseases": ["yes"],
    "Accident or serious trauma": ["yes"],
    "Surgical intervention": ["yes"],
    "High fevers in the last year": ["more than 3 months ago"],
    "Frequency of alcohol consumption": ["once a week"],
    "Smoking habit": ["never"],
    "Age": [30],
    "Hours Sitting per Day": [8]
}
data = pd.DataFrame(data)

#fromApiVal.getPred(data)

The request failed with status code: 424
server: azureml-frontdoor
date: Thu, 27 Jul 2023 19:41:53 GMT
content-type: application/json
content-length: 92
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/0.8.4
x-ms-request-id: ef0f14ad-aa14-439e-af07-7c26a708d99b
x-request-id: ef0f14ad-aa14-439e-af07-7c26a708d99b
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: automl4f78aa81750-1
connection: close


{"message": "An unexpected error occurred in scoring script. Check the logs for more info."}


'424'

In [90]:
data = {
    "Season": ["spring"],
    "Childish diseases": ["yes"],
    "Accident or serious trauma": ["yes"],
    "Surgical intervention": ["yes"],
    "High fevers in the last year": ["more than 3 months ago"],
    "Frequency of alcohol consumption": ["once a week"],
    "Smoking habit": ["never"],
    "Age": [30],
    "Hours Sitting per Day": [8]
}
data = pd.DataFrame(data)

In [91]:
fromApiVal = API_values()

In [1]:
data_processed = fromApiVal.transform(data) 

NameError: name 'fromApiVal' is not defined

In [84]:
data_processed

,Season,Childish diseases,Accident or serious trauma,Surgical intervention,High fevers in the last year,Frequency of alcohol consumption,Smoking habit,Age,Hours Sitting per Day
0,0,1,1,1,2,1.0,2,30,8


In [85]:
data_json = data_processed.to_json(orient='records')
data_json

'[{"Season":0,"Childish diseases":1,"Accident or serious trauma":1,"Surgical intervention":1,"High fevers in the last year":2,"Frequency of alcohol consumption":1.0,"Smoking habit":2,"Age":30,"Hours Sitting per Day":8}]'

In [86]:
fromApiVal.getPred(data)

The request failed with status code: 424
server: azureml-frontdoor
date: Thu, 27 Jul 2023 19:37:44 GMT
content-type: application/json
content-length: 92
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/0.8.4
x-ms-request-id: 3ef999ce-5525-4814-a038-c3973dff7644
x-request-id: 3ef999ce-5525-4814-a038-c3973dff7644
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: automl4f78aa81750-1
connection: close


{"message": "An unexpected error occurred in scoring script. Check the logs for more info."}


'424'